# Dynamic Credit Line Adjustment System  
## Phase 1: Probability of Default (PD) Model  

Author: Rudransh Shukla  
Objective: Predict Probability of Default (PD) using Logistic Regression  
Framework: CRISP-DM


## 1. Business Understanding

Goal:
Build a Probability of Default (PD) model to support a dynamic credit line adjustment system.

Business Use:
- Identify high-risk customers
- Reduce exposure for risky accounts
- Increase limits for low-risk accounts

Evaluation Metric:
- ROC-AUC


In [60]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


load the dataset

In [62]:
dataset_path = '/content/drive/MyDrive/phase2_clean.parquet'

try:
    df = pd.read_parquet(dataset_path)
    print(f"Dataset loaded successfully from: {dataset_path}")
except FileNotFoundError:
    print(f"Error: The file '{dataset_path}' was not found. Please ensure the file is uploaded to your Google Drive and the path is correct.")
except Exception as e:
    print(f"An error occurred while loading the dataset: {e}")

Dataset loaded successfully from: /content/drive/MyDrive/phase2_clean.parquet


now we check for data inconsistency, see if there are any extreme outliers

In [63]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
loan_amnt,1345310.0,14419.972014,8717.050787,500.00,8000.00,12000.000000,20000.00,4.000000e+04
term,1345310.0,41.790196,10.268321,36.00,36.00,36.000000,36.00,6.000000e+01
int_rate,1345310.0,13.239619,4.768716,5.31,9.75,12.740000,15.99,3.099000e+01
installment,1345310.0,438.075533,261.512604,4.93,248.48,375.430000,580.73,1.719830e+03
revol_util,1344453.0,51.810022,24.521072,0.00,33.40,52.200000,70.70,8.923000e+02
revol_bal,1345310.0,16248.114860,22328.168731,0.00,5943.00,11134.000000,19755.75,2.904836e+06
bc_util,1283398.0,59.939572,28.293216,0.00,38.30,63.200000,84.80,3.396000e+02
all_util,537545.0,58.176372,20.935438,0.00,44.00,59.000000,73.00,1.980000e+02
total_rev_hi_lim,1277783.0,32790.937175,36642.589282,0.00,14000.00,24100.000000,40600.00,9.999999e+06
bc_open_to_buy,1284167.0,10186.234200,15350.837521,0.00,1464.00,4700.000000,12383.00,5.599120e+05


data preprocessing

In [64]:
(df['revol_util'] > 200).sum()

np.int64(2)

In [65]:
df.loc[df['revol_util'] > 100 , 'revol_util'] = np.nan

In [66]:
df.loc[df['bc_util'] > 100 , 'bc_util'] = np.nan

In [67]:
df.loc[df['all_util'] > 100 , 'all_util'] = np.nan

In [68]:
df.loc[df['total_rev_hi_lim'] == 9999999, 'total_rev_hi_lim'] = np.nan
df.loc[df['tot_hi_cred_lim'] == 9999999, 'tot_hi_cred_lim'] = np.nan

now looking at our df.describe().T we see that annual income is very high so we check for the 99.9% of the data

In [69]:
df['annual_inc'].quantile(0.999)

np.float64(577382.0000002161)

In [70]:
income_cap = df['annual_inc'].quantile(0.999)

df.loc[df['annual_inc'] > income_cap, 'annual_inc'] = income_cap


In [71]:
df['revol_bal'].quantile(0.999)

np.float64(263117.8930000025)

In [72]:
revol_bal_cap = df['revol_bal'].quantile(0.999)

df.loc[df['revol_bal'] > revol_bal_cap , 'revol_bal'] = revol_bal_cap

In [73]:
(df['annual_inc'] == 0).sum()

np.int64(361)

In [74]:
df.loc[df['annual_inc'] == 0 , 'annual_inc'] = np.nan

In [75]:
df['total_rev_hi_lim'].quantile(0.999)

np.float64(342543.9120000098)

In [76]:
total_limit_cap = df['total_rev_hi_lim'].quantile(0.999)

In [77]:
df.loc[df['total_rev_hi_lim'] > total_limit_cap , 'total_rev_hi_lim'] = np.nan

In [78]:
df.describe(percentiles=[0.9, 0.95, 0.99, 0.999]).T

,count,mean,std,min,50%,90%,95%,99%,99.9%,max
loan_amnt,1345310.0,14419.972014,8717.050787,500.00,12000.000000,28000.000000,32875.000000,35000.000000,4.000000e+04,4.000000e+04
term,1345310.0,41.790196,10.268321,36.00,36.000000,60.000000,60.000000,60.000000,6.000000e+01,6.000000e+01
int_rate,1345310.0,13.239619,4.768716,5.31,12.740000,19.520000,22.150000,26.300000,3.089000e+01,3.099000e+01
installment,1345310.0,438.075533,261.512604,4.93,375.430000,807.920000,963.660000,1221.500000,1.368660e+03,1.719830e+03
revol_util,1339766.0,51.629353,24.359060,0.00,52.000000,84.600000,91.000000,97.500000,9.970000e+01,1.000000e+02
revol_bal,1345310.0,16125.862924,19748.077832,0.00,11134.000000,32425.000000,43374.000000,94553.910000,2.631130e+05,2.631179e+05
bc_util,1262137.0,59.218516,27.961860,0.00,62.400000,94.500000,97.200000,99.300000,1.000000e+02,1.000000e+02
all_util,530951.0,57.540089,20.240626,0.00,59.000000,83.000000,88.000000,97.000000,1.000000e+02,1.000000e+02
total_rev_hi_lim,1276502.0,32326.125362,29789.381564,0.00,24000.000000,64200.000000,85200.000000,151800.000000,2.782499e+05,3.425280e+05
bc_open_to_buy,1284167.0,10186.234200,15350.837521,0.00,4700.000000,26299.400000,38995.700000,74182.680000,1.346504e+05,5.599120e+05


In [79]:
cap = df['delinq_amnt'].quantile(0.999)
df.loc[df['delinq_amnt'] > cap, 'delinq_amnt'] = cap

In [80]:
df['delinq_amnt'].quantile([0.99, 0.995, 0.999])

,delinq_amnt
0.990,0.000000
0.995,0.000000
0.999,1073.864886


In [81]:
# now we freeze the data set

In [82]:
df.drop('loan_status',axis = 1,inplace= True)

In [83]:
df_clean = df.copy()

In [84]:
df_clean.to_parquet("/content/drive/MyDrive/pd_cleaned.parquet", index=False)

In [85]:
X = df_clean.drop("default", axis=1)
y = df_clean["default"]


In [86]:
y.value_counts(normalize=True)

,proportion
default,
0,0.800374
1,0.199626


After cleaning, separating numerical columns and categorical columns

In [87]:
num_cols = X.select_dtypes(include=np.number).columns
cat_cols = X.select_dtypes(exclude= np.number).columns

print("Numerical:", len(num_cols))
print("Categorical:", len(cat_cols))


Numerical: 45
Categorical: 4


In [88]:
cat_cols

Index(['purpose', 'verification_status', 'home_ownership', 'addr_state'], dtype='object')

Building the Piplines for numerical and categorical data preprocessing

In [89]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ("numerical",num_pipeline, num_cols),
    ('categorical',cat_pipeline, cat_cols)
])

building the model

In [90]:
model = LogisticRegression(
    max_iter= 1000,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

In [91]:
log_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

In [92]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (1076248, 49)
Test shape: (269062, 49)


In [93]:
X_train_sample = X_train.sample(300000, random_state=42)
y_train_sample = y_train[X_train_sample.index]

In [94]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

Cross Validation for better results on unseen data

In [95]:
skf = StratifiedKFold(n_splits=3, shuffle = True, random_state=42)

log_cross_score = cross_val_score(
    log_pipe,
    X_train_sample,
    y_train_sample,
    cv=skf,
    scoring='roc_auc',
    n_jobs= 1,
    verbose=True
)

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   35.4s finished


In [96]:
print("CV AUCs:", log_cross_score)
print("Mean CV AUC:", log_cross_score.mean())

CV AUCs: [0.7139044  0.71623355 0.71384336]
Mean CV AUC: 0.7146604367382263


In [97]:
log_pipe.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['loan_amnt', 'term', 'int_rate', 'installment', 'revol_util',
       'revol_bal', 'bc_util', 'all_util', 'total_rev_hi_lim',
       'bc_open_to_buy', 'max_bal_bc', 'total_bc_limit', 'total_bal_ex_mort'...
       'credit_age'],
      dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['purpose', 'verification_status', 'home_ownership', 'addr_state'], dtype='object'))])),
                ('model',
                 LogisticRegression(class_weight='balanced', max_iter=1000,
                                    n_jobs=-1, random_state=42))])

In [98]:
test_prob = log_pipe.predict_proba(X_test)[:,1]

In [99]:
test_auc = roc_auc_score(y_test,test_prob)

In [100]:
print("Final Test AUC:", test_auc)

Final Test AUC: 0.7139224255925487


now our model gives probabilities for default or PD but it doesn't give us the financial decision so we define some features like:-
1. ead:- Exposure At Default(the amount exposed to the bank or lending company after default)
2. pd_score:- the probability score
3. actual default:- actual outcome
4. risk bucket:- segmenting pd_score in 5 sections to find the connection between pd_score and actual default
5. lgd :- loss given default
6. expected_loss:- the expected loss on an individual for the bank for lending money which is basically (ead * lgd * pd_score)

In [101]:
X_test_with_pd = X_test.copy()
X_test_with_pd['pd_score'] = test_prob
X_test_with_pd['actual_default'] = y_test.values

X_test_with_pd[['pd_score','actual_default']].head()

,pd_score,actual_default
1311262,0.372454,0
823606,0.339770,0
196043,0.404968,0
810959,0.393673,0
1157129,0.469992,0


In [102]:
X_test_with_pd['risk_bucket'] = pd.qcut(
    X_test_with_pd['pd_score'],
    q=5,
    labels=['Very Low','Low','Medium','High','Very High']
)

In [103]:
X_test_with_pd['risk_bucket'].value_counts()

,count
risk_bucket,
Very Low,53813
Very High,53813
Low,53812
Medium,53812
High,53812


In [104]:
bucket_analysis = X_test_with_pd.groupby('risk_bucket')['actual_default'].mean()
bucket_analysis

/tmp/ipython-input-3765258614.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bucket_analysis = X_test_with_pd.groupby('risk_bucket')['actual_default'].mean()


,actual_default
risk_bucket,
Very Low,0.058220
Low,0.117167
Medium,0.174292
High,0.251078
Very High,0.397376


In [105]:
X_test_with_pd['ead'] = X_test_with_pd['total_rev_hi_lim']

In [106]:
lgd = 0.6  #assuming that loss given default is 60% as organization is able to recover 40% of the cost

In [107]:
X_test_with_pd['expected_loss'] = X_test_with_pd['pd_score']*X_test_with_pd['ead']*lgd

In [108]:
baseline_portfolio_el = X_test_with_pd['expected_loss'].sum()
baseline_portfolio_el

np.float64(2082780556.0732586)

adding a limit adjustment logic to reduce the expected loss on a person

In [109]:
def adjust_limit(bucket):
    if bucket == 'Very Low':
        return 1.10
    elif bucket == 'Low':
        return 1.05
    elif bucket == 'Medium':
        return 1.00
    elif bucket == 'High':
        return 0.80
    else:
        return 0.60

X_test_with_pd['limit_multiplier'] = (
    X_test_with_pd['risk_bucket'].apply(adjust_limit)
)


calculating the expected loss again with adjusted limit

In [110]:
X_test_with_pd['new_ead'] = (
    X_test_with_pd['ead'] * X_test_with_pd['limit_multiplier'].astype(float)
)

X_test_with_pd['adjusted_EL'] = (
    X_test_with_pd['pd_score'] * lgd * X_test_with_pd['new_ead']
)

adjusted_portfolio_EL = X_test_with_pd['adjusted_EL'].sum()

print("Adjusted Portfolio EL:", adjusted_portfolio_EL)


Adjusted Portfolio EL: 1789960862.8499682


In [111]:
reduction = baseline_portfolio_el - adjusted_portfolio_EL
reduction_per = (reduction/baseline_portfolio_el)*100

In [112]:
reduction_per

np.float64(14.059075612620179)

Conclusion:

A baseline PD model with AUC ~0.71 was developed using logistic regression.
Risk segmentation showed clear monotonic default behavior.
Applying a PD-driven dynamic limit adjustment policy reduced simulated portfolio expected loss by 14%.

Future Work:
- Hyperparameter tuning
- Model calibration
- Revenue-risk optimization
- Advanced models (XGBoost)
